In [11]:
import pandas as pd

# 读取JSON文件并转换为DataFrame
test_datadf = pd.read_json('../data/unique_sentences.json')

# 显示DataFrame的前几行数据
print(test_datadf.head())

# 将DataFrame中的数据存储到一个列表中
test_data_list = test_datadf.values.tolist()

# 打印列表中的数据
print(len(test_data_list))

                                             content
0  我家对面的门前有一大丛漂亮的花，它们在阳光下竞相开放，五颜六色。但蝴蝶花在这些花中脱颖而出，...
1  露岚也来到了坝子上，静静地浮着，不再回到山谷里去。阳光虽然依旧明亮，却不再痛炙人的脊梁，变得...
2        秋风飒飒地刮过之后，地上铺满了厚厚的落叶，踩上去“沙沙”作响，更增添了小院浓浓的秋意。
3  周围起起伏伏的山峦披着绿的清新。向下看去，一层层错落有致的梯田一片片绿色，深绿和翠绿交错着。...
4  那绿缎般清澈的金沙江水，衬托着多姿多彩又美丽宁静的小村庄，缓缓流淌，发出哗哗的轻快的响声。这...
223


In [12]:
AUTHOR_PROMPT = "根据故事："
TITLE_PROMPT = "作诗一首"
EOS_TOKEN = '</s>'

In [13]:
def poem(title_str, opt_story):
  in_request = TITLE_PROMPT + title_str[:12] + EOS_TOKEN + AUTHOR_PROMPT + opt_story[:300]

  return in_request

In [14]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)


def getOneMessageGPT(prompt):

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4",
    )
    return chat_completion.choices[0].message.content

In [15]:
gpt_result = []
prompts = []
for title in ['七言律诗', "七言绝句", '五言律诗',"五言绝句"]:
    # Empty author means general style
    for story in test_data_list[:1]:
        prompts.append(poem(title, story[0]))
        gpt_result.append(getOneMessageGPT(poem(title, story[0])))

In [16]:
prompts

['作诗一首七言律诗</s>根据故事：我家对面的门前有一大丛漂亮的花，它们在阳光下竞相开放，五颜六色。但蝴蝶花在这些花中脱颖而出，宛如一颗钻石在花丛中闪耀着他的光芒。',
 '作诗一首七言绝句</s>根据故事：我家对面的门前有一大丛漂亮的花，它们在阳光下竞相开放，五颜六色。但蝴蝶花在这些花中脱颖而出，宛如一颗钻石在花丛中闪耀着他的光芒。',
 '作诗一首五言律诗</s>根据故事：我家对面的门前有一大丛漂亮的花，它们在阳光下竞相开放，五颜六色。但蝴蝶花在这些花中脱颖而出，宛如一颗钻石在花丛中闪耀着他的光芒。',
 '作诗一首五言绝句</s>根据故事：我家对面的门前有一大丛漂亮的花，它们在阳光下竞相开放，五颜六色。但蝴蝶花在这些花中脱颖而出，宛如一颗钻石在花丛中闪耀着他的光芒。']

In [17]:
gpt_result

['门前百花竞相妍，阳光之下显彩饰。\n五颜扮绿叶，六色灿烂逼日石。\n独立花丛中，蝴蝶花娇艳笑。\n光彩夺目照，如玉辉煌透花扉。',
 '门前花影艳阳天，五色斑斓映日边。\n蝴蝶花中独璀璨，花丛钻石闪光环。',
 '对门花丛压路旁，阳光洒落竞争艳。\n色彩斑斓喜春光，蝴蝶花中独放光。\n如诗如画一花园，借古襄今人欣欣。\n艳似一颗璀璨钻，闪烁花丛映日辉。',
 '花丛争艳耀日辉，蝶舞独秀披钻彩。\n对门灿烂生晴意，一幕春华画里开。']

# 百度 文心一言

In [18]:
API_KEY = "BD6Os72JDhVbQfI51ekevUQU"
SECRET_KEY = "SD0fjV3OT5g5N9wykHcIsIu3dUPFnmhJ"

In [19]:
import requests
import json

def get_access_token():
    """
    使用 AK，SK 生成鉴权签名（Access Token）
    :return: access_token，或是None(如果错误)
    """
    url = "https://aip.baidubce.com/oauth/2.0/token"
    params = {"grant_type": "client_credentials", "client_id": API_KEY, "client_secret": SECRET_KEY}
    return str(requests.post(url, params=params).json().get("access_token"))

In [20]:
access_token = get_access_token()
access_token

'24.726787b435a4d64cf5db573c246a855e.2592000.1719295996.282335-74556683'

In [21]:
def getOneMessageWXYY(prompt):
    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/eb-instant?access_token=" + access_token
    # 注意message必须是奇数条
    payload = json.dumps({
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
            #,
            #{
            #    "role": "assistant",
            #    "content": "你好，有什么我可以帮助你的吗？"
            #}
        ]
    })
    headers = {
        'Content-Type': 'application/json'
    }
 
    response = requests.request("POST", url, headers=headers, data=payload)
    dict = json.loads(response.text)
    return dict['result']

In [22]:
getOneMessageWXYY("作诗一首七言绝句</s>根据故事：周围起起伏伏的山峦披着绿的清新。向下看去，一层层错落有致的梯田一片片绿色，深绿和翠绿交错着。我感叹到：“啊!真美呀!”在山谷的底部，一条条小溪纵横交错着。")

'山峦起伏披新绿，\n梯田层叠翠深交。\n小溪纵横山谷底，\n美景令人叹不休。'

# 我们的模型

In [23]:
import torch
from simplet5 import SimpleT5
from transformers import T5Tokenizer, T5ForConditionalGeneration
# Load model directly
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 指定模型和分词器的路径
model_path = "../output/simplet5-epoch-2-train-loss-3.8454-val-loss-4.0087"
tokenizer_path = "../output/simplet5-epoch-2-train-loss-3.8454-val-loss-4.0087"
torch.cuda.empty_cache()
class testMengziSimpleT5(SimpleT5):
  def __init__(self) -> None:
    super().__init__()
    self.device = torch.device("cuda")

  def load_my_model(self, use_gpu: bool = True):
    # 加载分词器
    self.tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)

    # 加载模型
    self.model = T5ForConditionalGeneration.from_pretrained(model_path)



model_test = testMengziSimpleT5()
model_test.load_my_model()
model_test.model = model_test.model.to('cuda')

Global seed set to 42


In [24]:
def getOneMessageModel(in_request, model=model_test, num_beams=2):
  model.model = model.model.to('cuda')
  out = model.predict(in_request,
                      max_length=64,
                      num_beams=num_beams)[0].replace(",", "，")

  return out

In [25]:
getOneMessageModel('作诗一首七言绝句</s>根据故事：周围起起伏伏的山峦披着绿的清新。向下看去，一层层错落有致的梯田一片片绿色，深绿和翠绿交错着。我感叹到：“啊!真美呀!”在山谷的底部，一条条小溪纵横交错着。')

'叠叠层层叠嶂间，层层重叠映青山。 俯看梯田片青绿，深绿和烟覆绿斑。'

# 整体测试

In [26]:
def generateOne(title, story):
    prompt = poem(title, story)
    result_gpt4 = getOneMessageGPT(prompt).replace(" ", "").replace("\n", "")
    result_wxyy = getOneMessageWXYY(prompt).replace(" ", "").replace("\n", "")
    result_model = getOneMessageModel(prompt).replace(" ", "").replace("\n", "")
    return {'prompt':prompt, 'gpt4-result':result_gpt4, 'wenxinyiyan-result':result_wxyy, 'our-result': result_model}

In [27]:
def testModels(num_poem=2):
    results = []
    
    # for title in ['七言律诗', "七言绝句", '五言律诗',"五言绝句"]:
    for title in ['七言律诗', "五言绝句"]:
        # Empty author means general style
        for story in test_data_list[:num_poem]:
            story = story[0]
            results.append(generateOne(title, story))
    
    return results

In [34]:
generateOne('七言绝句', '我最好的朋友为官清正，忠心耿耿，可惜朝中奸臣嫉妒他的才能，以莫须有的罪名害死了他。我抱着他的遗物来到他的故乡，看着乡亲们期盼见到他的眼神，我不禁痛哭流涕。')

{'prompt': '作诗一首七言绝句</s>根据故事：我最好的朋友为官清正，忠心耿耿，可惜朝中奸臣嫉妒他的才能，以莫须有的罪名害死了他。我抱着他的遗物来到他的故乡，看着乡亲们期盼见到他的眼神，我不禁痛哭流涕。',
 'gpt4-result': '清官悲煞奸臣恶，忠心枉死泪沾衣。归乡抱遗亲罔识，痛入骨髓泣斜晖。',
 'wenxinyiyan-result': '七言绝句故乡晴空照我哀，遗物持手中泪淹。忠贞耿介谢世事，遗恨奸臣祸满台。',
 'our-result': '我有一友皆清直，岂料朝中妒莫须。抱余遗物还乡国，看取乡亲欲见无。'}

In [38]:
generateOne('七言律诗', '我最好的朋友为官清正，忠心耿耿，可惜朝中奸臣嫉妒他的才能，以莫须有的罪名害死了他。我抱着他的遗物来到他的故乡，看着乡亲们期盼见到他的眼神，我不禁痛哭流涕。')

{'prompt': '作诗一首七言律诗</s>根据故事：我最好的朋友为官清正，忠心耿耿，可惜朝中奸臣嫉妒他的才能，以莫须有的罪名害死了他。我抱着他的遗物来到他的故乡，看着乡亲们期盼见到他的眼神，我不禁痛哭流涕。',
 'gpt4-result': '朋友德官清如洁，忠义侠骨映朝阳。奸臣嫉妒欲摧折，莫须有罪归黄土。遗象怀中泪满胆，长驱千里到故乡。乡亲眼望空归客，悲痛欲绝泣乾坤。',
 'wenxinyiyan-result': '好的，以下是根据您的要求写的一首七言律诗：清官忠骨恨奸臣，莫须有罪枉死身。遗物携至故乡来，期盼眼神痛心扉。朝野奸佞妒才子，清廉声名枉成灰。痛哭流涕思故友，英魂永驻故乡水。',
 'our-result': '我友清和志尚坚，岂知朝士恨何须。君王若妒奸臣才，我辈何须怨子胥。倚玉遗书来帝里，看花望月到乡乡。抚膺一顾思乡泪，看着乡亲欲泪垂。'}

In [29]:
generateOne('七言律诗', '早上我打开窗户，看到花园中凋谢了很多花，它们都是我的朋友，我非常伤心。我想到自己活在世上，和花园中的一朵花也没什么区别。我的盛开和凋零都不是自己能做主的事情，我只能追寻心灵的自由')

{'prompt': '作诗一首七言律诗</s>根据故事：早上我打开窗户，看到花园中凋谢了很多花，它们都是我的朋友，我非常伤心。我想到自己活在世上，和花园中的一朵花也没什么区别。我的盛开和凋零都不是自己能做主的事情，我只能追寻心灵的自由',
 'gpt4-result': '晨起我掀窗扉，望见园中花凋微。心怀友人情更深，玫瑰凋败伤欲绝。我也如其不自由，生之花开谁主权？尽寻心灵之自在，盛衰由命不由天。',
 'wenxinyiyan-result': '清晨打开窗，花园花凋零，朋友皆凋谢，伤心难自抑。人生如花事，盛开亦凋零，自由在心间，追寻方不虚。世事如浮云，生命似晨露，花开自有期，凋零亦有时。心若自由翔，何惧花落去，只愿心灵净，自由永无止。',
 'our-result': '晓开窗户见花频，看却庭花几处新。我到人间何足恨，与谁家世有何人。花间自是身闲事，花外何殊花有春。若要花开兼作主，只缘空逐梦中身。'}

In [37]:
generateOne('七言绝句', '独行的侠客走在绝壁上，风吹起长袍猎猎作响，周围传来几声鸟鸣。')

{'prompt': '作诗一首七言绝句</s>根据故事：独行的侠客走在绝壁上，风吹起长袍猎猎作响，周围传来几声鸟鸣。',
 'gpt4-result': '侠客独行绝壁头，风吹衣袂猎猎鸣。周围鸟语声娇柔，无言告别人间情。',
 'wenxinyiyan-result': '独行侠客绝壁行，长袍猎猎风声起。几声鸟鸣空谷响，山川壮丽入诗心。',
 'our-result': '侠客横戈指剑门，风来长袍射雕鸣。风飘短笠声犹在，风起长袍数鸟鸣。'}

In [30]:
results = testModels(1)

In [31]:
results

[{'prompt': '作诗一首七言律诗</s>根据故事：我家对面的门前有一大丛漂亮的花，它们在阳光下竞相开放，五颜六色。但蝴蝶花在这些花中脱颖而出，宛如一颗钻石在花丛中闪耀着他的光芒。',
  'gpt4-result': '门前花丛竞斗艳，阳光下显辉煌。五色占尽春色艳，蝴蝶花独秀扬。犹如钻石光闪耀，披星戴月争芳香。万千花朵皆成背影，独领风骚举世光。',
  'wenxinyiyan-result': '好的，以下是我为您创作的一首七言律诗：对面花丛艳阳天，蝶舞蜂鸣自欢颜。群芳争艳谁争锋，独蝶翩翩花间行。万紫千红皆寻常，唯有蝶花相映光。我家对面门前景，犹胜春色满园香。',
  'our-result': '我家西郭两丛花，春日相开照面新。繁艳竞开争发日，娇姿竞发竞生春。蝶飞蜂散成千朵，蝶舞蜂多出五色。百花中出百花里，一朵明珠万丈新。'},
 {'prompt': '作诗一首五言绝句</s>根据故事：我家对面的门前有一大丛漂亮的花，它们在阳光下竞相开放，五颜六色。但蝴蝶花在这些花中脱颖而出，宛如一颗钻石在花丛中闪耀着他的光芒。',
  'gpt4-result': '我家对门繁花笑，竞艳阳光色更饶。蝴蝶花中独放晴，犹如凡丛闪瑞兆。',
  'wenxinyiyan-result': '我家对面门，花开竞艳阳。蝴蝶花独秀，璀璨如钻石。',
  'our-result': '我家东郭花，日日竞相开。蝶在花中出，犹如一聚灰。'}]

In [32]:
with open('../output/model3results.jsonl', mode='w', encoding='utf8') as f:
    for item in results:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')